<a href="https://colab.research.google.com/github/oaken-one/COS_IW_06_oe7003/blob/main/Independent_Work_Code_Using_Machine_Learning_Models_to_Predict_Discharging_Capacity_in_Lithium_Ion_Batteries_OkezieEze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
## Uploading MATLAB files
import scipy.io as sio
bat05 = sio.loadmat("/content/drive/MyDrive/NASA Battery Data/B0005.ordered.mat")
bat06 = sio.loadmat("/content/drive/MyDrive/NASA Battery Data/B0006.ordered.mat")

In [4]:
## Preprocessing Constant Current Charging Voltage
import numpy as np
import pandas as pd

ccvB0005 = np.zeros([167, 5])
ccvB0006 = np.zeros([167, 5])

## Find the maximum and minimum values of Constant Current Charging Voltage
ccvB0005_max = bat05['B0005'][0,0]['cycle'][0,0]['data'][0,0]['Voltage_measured'].max()
ccvB0005_min = bat05['B0005'][0,0]['cycle'][0,0]['data'][0,0]['Voltage_measured'].min()

ccvB0006_max = bat06['B0006'][0,0]['cycle'][0,0]['data'][0,0]['Voltage_measured'].max()
ccvB0006_min = bat06['B0006'][0,0]['cycle'][0,0]['data'][0,0]['Voltage_measured'].min()

for x in range(167):
  if ccvB0005_max < bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max():
    ccvB0005_max = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max()

for x in range(167):
  if ccvB0006_max < bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max():
    ccvB0006_max = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max()

for x in range(167):
  if ccvB0005_min > bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min():
    ccvB0005_min = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min()

for x in range(167):
  if ccvB0006_min > bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min():
    ccvB0006_min = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min()

## Choose specific points for training models and normalize them
for x in range(167):
  ccvB0005[x, 0] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 3] - ccvB0005_min)/(ccvB0005_max - ccvB0005_min)
  ccvB0005[x, 1] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 36] - ccvB0005_min)/(ccvB0005_max - ccvB0005_min)
  ccvB0005[x, 2] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 69] - ccvB0005_min)/(ccvB0005_max - ccvB0005_min)
  ccvB0005[x, 3] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 127] - ccvB0005_min)/(ccvB0005_max - ccvB0005_min)
  ccvB0005[x, 4] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 249] - ccvB0005_min)/(ccvB0005_max - ccvB0005_min)

for x in range(167):
  ccvB0006[x, 0] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 3] - ccvB0006_min)/(ccvB0006_max - ccvB0006_min)
  ccvB0006[x, 1] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 36] - ccvB0006_min)/(ccvB0006_max - ccvB0006_min)
  ccvB0006[x, 2] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 69] - ccvB0006_min)/(ccvB0006_max - ccvB0006_min)
  ccvB0006[x, 3] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 127] - ccvB0006_min)/(ccvB0006_max - ccvB0006_min)
  ccvB0006[x, 4] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'][0, 249] - ccvB0006_min)/(ccvB0006_max - ccvB0006_min)

In [5]:
## Preprocessing Constant Voltage Charging Current
cvccB0005 = np.zeros([167, 5])
cvccB0006 = np.zeros([167, 5])

## Find the maximum and minimum values of Constant Voltage Charging Current
cvccB0005_max = bat05['B0005'][0,0]['cycle'][0,0]['data'][0,0]['Current_measured'].max()
cvccB0005_min = bat05['B0005'][0,0]['cycle'][0,0]['data'][0,0]['Current_measured'].min()

cvccB0006_max = bat06['B0006'][0,0]['cycle'][0,0]['data'][0,0]['Current_measured'].max()
cvccB0006_min = bat06['B0006'][0,0]['cycle'][0,0]['data'][0,0]['Current_measured'].min()

for x in range(167):
  if cvccB0005_max < bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].max():
    cvccB0005_max = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].max()

for x in range(167):
  if cvccB0006_max < bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].max():
    cvccB0006_max = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].max()

for x in range(167):
  if cvccB0005_min > bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].min():
    cvccB0005_min = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].min()

for x in range(167):
  if cvccB0006_min > bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].min():
    cvccB0006_min = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'].min()

## Choose specific points for training models and normalize them (cycle 32 does not have the same shape as other cycles and must be handled differently)
for x in range(167):
  if x == 32:
    cvccB0005[x, 0] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 4] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 1] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 8] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 2] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 13] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 3] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 28] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 4] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 32] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
  if x != 32:
    cvccB0005[x, 0] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 518] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 1] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 535] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 2] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 557] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 3] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 573] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)
    cvccB0005[x, 4] = (bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 594] - cvccB0005_min)/(cvccB0005_max - cvccB0005_min)

for x in range(167):
  if x == 32:
    cvccB0006[x, 0] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 4] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 1] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 8] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 2] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 13] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 3] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 28] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 4] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 32] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
  if x != 32:
    cvccB0006[x, 0] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 518] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 1] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 535] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 2] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 557] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 3] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 573] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)
    cvccB0006[x, 4] = (bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Current_measured'][0, 594] - cvccB0006_min)/(cvccB0006_max - cvccB0006_min)

In [6]:
## Preprocessing Constant Current Charging Time
ccctB0005 = np.zeros([167, 1])
ccctB0006 = np.zeros([167, 1])

for x in range(167):
  max_voltage_index_B5 = np.argmax(bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'])
  ccctB0005[x,0] = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Time'][0, max_voltage_index_B5]

for x in range(167):
  max_voltage_index_B6 = np.argmax(bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'])
  ccctB0006[x,0] = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Time'][0, max_voltage_index_B6]

## Find the maximum and minimum values of Constant Current Charging Time
ccct_B5_max = ccctB0005.max()
ccct_B5_min = ccctB0005.min()

ccct_B6_max = ccctB0006.max()
ccct_B6_min = ccctB0006.min()

## Choose specific points for training models and normalize them
for x in range(167):
  ccctB0005[x,0] = (ccctB0005[x,0] - ccct_B5_min)/(ccct_B5_max - ccct_B5_min)
  ccctB0006[x,0] = (ccctB0006[x,0] - ccct_B6_min)/(ccct_B6_max - ccct_B6_min)

In [7]:
## Preprocessing Constant Voltage Discharging Current
ccdvB0005 = np.zeros([167, 5])
ccdvB0006 = np.zeros([167, 5])

## Find the maximum and minimum values of Constant Voltage Discharging Current
ccdvB0005_max = bat05['B0005'][0,0]['cycle'][0,170]['data'][0,0]['Voltage_measured'].max()
ccdvB0005_min = bat05['B0005'][0,0]['cycle'][0,170]['data'][0,0]['Voltage_measured'].min()

ccdvB0006_max = bat06['B0006'][0,0]['cycle'][0,170]['data'][0,0]['Voltage_measured'].max()
ccdvB0006_min = bat06['B0006'][0,0]['cycle'][0,170]['data'][0,0]['Voltage_measured'].min()

for x in range(170, 338):
  if ccdvB0005_max < bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max():
    ccdvB0005_max = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max()

for x in range(170, 338):
  if ccdvB0006_max < bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max():
    ccdvB0006_max = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].max()

for x in range(170, 338):
  if ccdvB0005_min > bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min():
    ccdvB0005_min = bat05['B0005'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min()

for x in range(170, 338):
  if ccdvB0006_min > bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min():
    ccdvB0006_min = bat06['B0006'][0,0]['cycle'][0,x]['data'][0,0]['Voltage_measured'].min()

## Choose specific points for training models and normalize them
for x in range(167):
  ccdvB0005[x, 0] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 99] - ccdvB0005_min)/(ccdvB0005_max - ccdvB0005_min)
  ccdvB0005[x, 1] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 110] - ccdvB0005_min)/(ccdvB0005_max - ccdvB0005_min)
  ccdvB0005[x, 2] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 115] - ccdvB0005_min)/(ccdvB0005_max - ccdvB0005_min)
  ccdvB0005[x, 3] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 121] - ccdvB0005_min)/(ccdvB0005_max - ccdvB0005_min)
  ccdvB0005[x, 4] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 126] - ccdvB0005_min)/(ccdvB0005_max - ccdvB0005_min)

for x in range(167):
  ccdvB0006[x, 0] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 99] - ccdvB0006_min)/(ccdvB0006_max - ccdvB0006_min)
  ccdvB0006[x, 1] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 110] - ccdvB0006_min)/(ccdvB0006_max - ccdvB0006_min)
  ccdvB0006[x, 2] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 115] - ccdvB0006_min)/(ccdvB0006_max - ccdvB0006_min)
  ccdvB0006[x, 3] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 121] - ccdvB0006_min)/(ccdvB0006_max - ccdvB0006_min)
  ccdvB0006[x, 4] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 126] - ccdvB0006_min)/(ccdvB0006_max - ccdvB0006_min)


In [8]:
## Preprocessing Constant Current Discharging Time
ccdtB0005 = np.zeros([167, 1])
ccdtB0006 = np.zeros([167, 1])

for x in range(167):
  max_voltage_index_B5 = np.argmax(bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'])
  ccdtB0005[x,0] = bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, max_voltage_index_B5]

for x in range(167):
  max_voltage_index_B6 = np.argmax(bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'])
  ccdtB0006[x,0] = bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, max_voltage_index_B6]

## Find the maximum and minimum values of Constant Current Discharging Time
ccdt_B5_max = ccdtB0005.max()
ccdt_B5_min = ccdtB0005.min()

ccdt_B6_max = ccdtB0006.max()
ccdt_B6_min = ccdtB0006.min()

## Choose specific points for training models and normalize them
for x in range(167):
  ccdtB0005[x,0] = (ccdtB0005[x,0] - ccdt_B5_min)/(ccdt_B5_max - ccdt_B5_min)
  ccdtB0006[x,0] = (ccdtB0006[x,0] - ccdt_B6_min)/(ccdt_B6_max - ccdt_B6_min)

In [9]:
## Preprocessing Rate of Change of Discharging Voltage
ccdv_roc_B0005 = np.zeros([167, 4])
ccdv_roc_B0006 = np.zeros([167, 4])

for x in range(167):
  ccdv_roc_B0005[x, 0] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 110] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 99])/(bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 110] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 99])
  ccdv_roc_B0005[x, 1] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 115] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 110])/(bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 115] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 110])
  ccdv_roc_B0005[x, 2] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 121] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 115])/(bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 121] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 115])
  ccdv_roc_B0005[x, 3] = (bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 126] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 121])/(bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 126] - bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 121])

  ccdv_roc_B0006[x, 0] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 110] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 99])/(bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 110] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 99])
  ccdv_roc_B0006[x, 1] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 115] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 110])/(bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 115] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 110])
  ccdv_roc_B0006[x, 2] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 121] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 115])/(bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 121] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 115])
  ccdv_roc_B0006[x, 3] = (bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 126] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Voltage_measured'][0, 121])/(bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 126] - bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Time'][0, 121])

## Find the maximum and minimum values of Rate of Change of Discharging Voltage
ccdv_roc_B0005_max = ccdv_roc_B0005.max()
ccdv_roc_B0005_min = ccdv_roc_B0005.min()

ccdv_roc_B0006_max = ccdv_roc_B0006.max()
ccdv_roc_B0006_min = ccdv_roc_B0006.min()

## Choose specific points for training models and normalize them
for x in range(167):
  ccdv_roc_B0005[x, 0] = (ccdv_roc_B0005[x, 0] - ccdv_roc_B0005_min)/(ccdv_roc_B0005_max - ccdv_roc_B0005_min)
  ccdv_roc_B0005[x, 1] = (ccdv_roc_B0005[x, 1] - ccdv_roc_B0005_min)/(ccdv_roc_B0005_max - ccdv_roc_B0005_min)
  ccdv_roc_B0005[x, 2] = (ccdv_roc_B0005[x, 2] - ccdv_roc_B0005_min)/(ccdv_roc_B0005_max - ccdv_roc_B0005_min)
  ccdv_roc_B0005[x, 3] = (ccdv_roc_B0005[x, 3] - ccdv_roc_B0005_min)/(ccdv_roc_B0005_max - ccdv_roc_B0005_min)

  ccdv_roc_B0006[x, 0] = (ccdv_roc_B0006[x, 0] - ccdv_roc_B0006_min)/(ccdv_roc_B0006_max - ccdv_roc_B0006_min)
  ccdv_roc_B0006[x, 1] = (ccdv_roc_B0006[x, 1] - ccdv_roc_B0006_min)/(ccdv_roc_B0006_max - ccdv_roc_B0006_min)
  ccdv_roc_B0006[x, 2] = (ccdv_roc_B0006[x, 2] - ccdv_roc_B0006_min)/(ccdv_roc_B0006_max - ccdv_roc_B0006_min)
  ccdv_roc_B0006[x, 3] = (ccdv_roc_B0006[x, 3] - ccdv_roc_B0006_min)/(ccdv_roc_B0006_max - ccdv_roc_B0006_min)

In [10]:
## Preprocesing Discharging Capacity
dcB0005 = np.zeros([167, 1])
dcB0006 = np.zeros([167, 1])
dcInputB0005 = np.zeros([167, 1])
dcInputB0006 = np.zeros([167, 1])

for x in range(167):
  dcB0005[x, 0] = bat05['B0005'][0,0]['cycle'][0,x + 170]['data'][0,0]['Capacity'][0,0]
  dcB0006[x, 0] = bat06['B0006'][0,0]['cycle'][0,x + 170]['data'][0,0]['Capacity'][0,0]

## Find the maximum and minimum values of Discharging Capacity
dcB0005_max = dcB0005.max()
dcB0006_max = dcB0006.max()
dcB0005_min = dcB0005.min()
dcB0006_min = dcB0006.min()

## Choose specific points for training models and normalize them
for x in range(167):
  dcInputB0005[x, 0] = (dcB0005[x, 0] - dcB0005_min)/(dcB0005_max - dcB0005_min)
  dcInputB0006[x, 0] = (dcB0006[x, 0] - dcB0006_min)/(dcB0006_max - dcB0006_min)

In [12]:
## Linear Regression Model with Case 1 (Inputs are CCCT and CCV)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.19272419774141278
MAE: 0.4049060749094907
R^2: -18.834480577934034


In [14]:
## Linear Regression Model with Case 2 (Inputs are Discharging Capacity and CVCC)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.003936459240739472
MAE: 0.04637880948447572
R^2: 0.5948737871461451


In [15]:
## Linear Regression Model with Case 3 (Inputs are Discharging Capacity, CCCT, and CCV)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.02313528327127691
MAE: 0.1222239532627117
R^2: -1.3810000616779852


In [16]:
## Linear Regression Model with Case 4 (Inputs are Discharging Capacity, CCCT, CCV, and CVCC)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.021423571272003963
MAE: 0.1143413805623242
R^2: -1.2048368252890111


In [17]:
## Linear Regression Model with Case 5 (Inputs are CCDT and CCDV)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.10433688432085234
MAE: 0.3020994735065836
R^2: -9.737976496344269


In [18]:
## Linear Regression Model with Case 6 (Inputs are CCDV and RoCV)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.004546387363834529
MAE: 0.055570130923738574
R^2: 0.5321021806056079


In [19]:
## Linear Regression Model with Case 7 (Inputs are Discharging Capacity, CCDT, and CCDV)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.00811091149302768
MAE: 0.07036350201336979
R^2: 0.16525419037595868


In [20]:
## Linear Regression Model with Case 8 (Inputs are Discharging Capacity, CCDT, CCDV, and RoCV)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Create a linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

MSE: 0.0007072429236683814
MAE: 0.02029747652312289
R^2: 0.9272131045412182


In [33]:
## Multilayer Perceptron (MLP) Model with Case 1 (Inputs are CCCT and CCV)
import numpy as np
import pandas as pd
import tensorflow
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 54ms/step - loss: 6.4042 - val_loss: 2.3185
Epoch 2/10000
5/5 [==============================] - 0s 18ms/step - loss: 7.1751 - val_loss: 2.3597
Epoch 3/10000
5/5 [==============================] - 0s 10ms/step - loss: 5.4033 - val_loss: 2.3637
Epoch 4/10000
5/5 [==============================] - 0s 11ms/step - loss: 6.7339 - val_loss: 2.2878
Epoch 5/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.9547 - val_loss: 2.1814
Epoch 6/10000
5/5 [==============================] - 0s 12ms/step - loss: 5.5348 - val_loss: 2.1265
Epoch 7/10000
5/5 [==============================] - 0s 11ms/step - loss: 5.7354 - val_loss: 2.0795
Epoch 8/10000
5/5 [==============================] - 0s 10ms/step - loss: 4.3561 - val_loss: 2.0334
Epoch 9/10000
5/5 [==============================] - 0s 12ms/step - loss: 3.9233 - val_loss: 1.9691
Epoch 10/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.5707 - val_loss: 1.9045

In [45]:
## Multilayer Perceptron Model (MLP) with Case 2 (Inputs are Discharging Capacity and CVCC)
import numpy as np
import pandas as pd
import tensorflow
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 80ms/step - loss: 5.6789 - val_loss: 3.3037
Epoch 2/10000
5/5 [==============================] - 0s 24ms/step - loss: 5.8355 - val_loss: 2.9512
Epoch 3/10000
5/5 [==============================] - 0s 21ms/step - loss: 5.6792 - val_loss: 2.6427
Epoch 4/10000
5/5 [==============================] - 0s 22ms/step - loss: 4.4884 - val_loss: 2.3719
Epoch 5/10000
5/5 [==============================] - 0s 20ms/step - loss: 4.9690 - val_loss: 2.1479
Epoch 6/10000
5/5 [==============================] - 0s 22ms/step - loss: 4.6391 - val_loss: 1.8734
Epoch 7/10000
5/5 [==============================] - 0s 17ms/step - loss: 4.7594 - val_loss: 1.6348
Epoch 8/10000
5/5 [==============================] - 0s 19ms/step - loss: 3.5669 - val_loss: 1.4248
Epoch 9/10000
5/5 [==============================] - 0s 17ms/step - loss: 3.9544 - val_loss: 1.2505
Epoch 10/10000
5/5 [==============================] - 0s 15ms/step - loss: 3.9679 - val_loss: 1.1044

In [53]:
## Multilayer Perceptron (MLP) Model with Case 3 (Inputs are Discharging Capacity, CCCT, and CCV)
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 52ms/step - loss: 5.9162 - val_loss: 3.3181
Epoch 2/10000
5/5 [==============================] - 0s 11ms/step - loss: 6.8631 - val_loss: 2.9942
Epoch 3/10000
5/5 [==============================] - 0s 10ms/step - loss: 6.7274 - val_loss: 2.6958
Epoch 4/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.8213 - val_loss: 2.4195
Epoch 5/10000
5/5 [==============================] - 0s 10ms/step - loss: 5.1171 - val_loss: 2.1680
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 5.0630 - val_loss: 1.9479
Epoch 7/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.9809 - val_loss: 1.7721
Epoch 8/10000
5/5 [==============================] - 0s 14ms/step - loss: 4.5106 - val_loss: 1.6478
Epoch 9/10000
5/5 [==============================] - 0s 12ms/step - loss: 3.8161 - val_loss: 1.5146
Epoch 10/10000
5/5 [==============================] - 0s 11ms/step - loss: 3.1912 - val_loss: 1.3848

In [70]:
## Multilayer Perceptron (MLP) Model with Case 4 (Inputs are Discharging Capacity, CCCT, CCV, and CVCC)
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 57ms/step - loss: 6.7433 - val_loss: 3.3535
Epoch 2/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.6252 - val_loss: 3.1904
Epoch 3/10000
5/5 [==============================] - 0s 18ms/step - loss: 7.3321 - val_loss: 2.9017
Epoch 4/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.7203 - val_loss: 2.6351
Epoch 5/10000
5/5 [==============================] - 0s 16ms/step - loss: 4.8141 - val_loss: 2.3786
Epoch 6/10000
5/5 [==============================] - 0s 13ms/step - loss: 5.8100 - val_loss: 2.0804
Epoch 7/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.8814 - val_loss: 1.7890
Epoch 8/10000
5/5 [==============================] - 0s 12ms/step - loss: 3.2896 - val_loss: 1.5390
Epoch 9/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.7619 - val_loss: 1.3435
Epoch 10/10000
5/5 [==============================] - 0s 11ms/step - loss: 3.0670 - val_loss: 1.1519

In [84]:
## Multilayer Perceptron (MLP) Model with Case 5 (Inputs are CCDT and CCDV)
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 59ms/step - loss: 7.0484 - val_loss: 2.8311
Epoch 2/10000
5/5 [==============================] - 0s 13ms/step - loss: 6.5054 - val_loss: 2.6480
Epoch 3/10000
5/5 [==============================] - 0s 13ms/step - loss: 5.3521 - val_loss: 2.4737
Epoch 4/10000
5/5 [==============================] - 0s 17ms/step - loss: 5.6010 - val_loss: 2.3621
Epoch 5/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.1409 - val_loss: 2.2419
Epoch 6/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.2671 - val_loss: 2.1177
Epoch 7/10000
5/5 [==============================] - 0s 17ms/step - loss: 4.2194 - val_loss: 1.9514
Epoch 8/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.0180 - val_loss: 1.8703
Epoch 9/10000
5/5 [==============================] - 0s 12ms/step - loss: 3.8904 - val_loss: 1.8193
Epoch 10/10000
5/5 [==============================] - 0s 12ms/step - loss: 3.9895 - val_loss: 1.7521

In [117]:
## Multilayer Perceptron (MLP) Model with Case 6 (Inputs are CCDV and RoCV)
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 82ms/step - loss: 5.9649 - val_loss: 2.9467
Epoch 2/10000
5/5 [==============================] - 0s 15ms/step - loss: 4.6577 - val_loss: 2.7250
Epoch 3/10000
5/5 [==============================] - 0s 17ms/step - loss: 6.0348 - val_loss: 2.5693
Epoch 4/10000
5/5 [==============================] - 0s 20ms/step - loss: 5.5280 - val_loss: 2.4485
Epoch 5/10000
5/5 [==============================] - 0s 17ms/step - loss: 6.1186 - val_loss: 2.2314
Epoch 6/10000
5/5 [==============================] - 0s 22ms/step - loss: 4.6439 - val_loss: 2.0513
Epoch 7/10000
5/5 [==============================] - 0s 17ms/step - loss: 4.2424 - val_loss: 1.8351
Epoch 8/10000
5/5 [==============================] - 0s 15ms/step - loss: 5.0113 - val_loss: 1.6884
Epoch 9/10000
5/5 [==============================] - 0s 18ms/step - loss: 3.5495 - val_loss: 1.5500
Epoch 10/10000
5/5 [==============================] - 0s 17ms/step - loss: 3.6717 - val_loss: 1.4122

In [144]:
## Multilayer Perceptron (MLP) Model with Case 7 (Inputs are Discharging Capacity, CCDT, and CCDV)
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 3s 75ms/step - loss: 7.3478 - val_loss: 2.6841
Epoch 2/10000
5/5 [==============================] - 0s 12ms/step - loss: 5.4225 - val_loss: 2.4343
Epoch 3/10000
5/5 [==============================] - 0s 11ms/step - loss: 5.9232 - val_loss: 2.3006
Epoch 4/10000
5/5 [==============================] - 0s 13ms/step - loss: 6.1004 - val_loss: 2.1249
Epoch 5/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.9437 - val_loss: 2.0101
Epoch 6/10000
5/5 [==============================] - 0s 11ms/step - loss: 5.1166 - val_loss: 1.8466
Epoch 7/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.4158 - val_loss: 1.7271
Epoch 8/10000
5/5 [==============================] - 0s 12ms/step - loss: 4.3922 - val_loss: 1.5869
Epoch 9/10000
5/5 [==============================] - 0s 17ms/step - loss: 3.4130 - val_loss: 1.4703
Epoch 10/10000
5/5 [==============================] - 0s 12ms/step - loss: 3.4030 - val_loss: 1.3665

In [158]:
## Multilayer Perceptron (MLP) Model with Case 8 (Inputs are Discharging Capacity, CCDT, CCDV, and RoCV)
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the KFold cross validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 10
for train_index, val_index in kfold.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

# MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10000, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
predictions = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 2s 52ms/step - loss: 6.1083 - val_loss: 3.5040
Epoch 2/10000
5/5 [==============================] - 0s 12ms/step - loss: 5.3143 - val_loss: 3.0876
Epoch 3/10000
5/5 [==============================] - 0s 11ms/step - loss: 5.8586 - val_loss: 2.7043
Epoch 4/10000
5/5 [==============================] - 0s 11ms/step - loss: 5.5608 - val_loss: 2.4156
Epoch 5/10000
5/5 [==============================] - 0s 12ms/step - loss: 5.2446 - val_loss: 2.1723
Epoch 6/10000
5/5 [==============================] - 0s 16ms/step - loss: 3.4584 - val_loss: 1.9182
Epoch 7/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.0899 - val_loss: 1.7210
Epoch 8/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.1873 - val_loss: 1.5326
Epoch 9/10000
5/5 [==============================] - 0s 16ms/step - loss: 3.7911 - val_loss: 1.3492
Epoch 10/10000
5/5 [==============================] - 0s 11ms/step - loss: 4.1525 - val_loss: 1.1921

In [174]:
## Convolutional Neural Network (CNN) with Case 1 (Inputs are CCCT and CCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 72ms/step - loss: 2.7282 - val_loss: 1.8765
Epoch 2/10000
5/5 [==============================] - 0s 19ms/step - loss: 2.1432 - val_loss: 1.2897
Epoch 3/10000
5/5 [==============================] - 0s 21ms/step - loss: 1.5294 - val_loss: 0.8424
Epoch 4/10000
5/5 [==============================] - 0s 21ms/step - loss: 1.0645 - val_loss: 0.5449
Epoch 5/10000
5/5 [==============================] - 0s 19ms/step - loss: 0.7220 - val_loss: 0.4033
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.4944 - val_loss: 0.3623
Epoch 7/10000
5/5 [==============================] - 0s 12ms/step - loss: 0.3307 - val_loss: 0.3636
Epoch 8/10000
5/5 [==============================] - 0s 15ms/step - loss: 0.2676 - val_loss: 0.3653
Epoch 9/10000
5/5 [==============================] - 0s 12ms/step - loss: 0.3062 - val_loss: 0.3537
Epoch 10/10000
5/5 [==============================] - 0s 12ms/step - loss: 0.1964 - val_loss: 0.3402

In [26]:
## Convolutional Neural Network (CNN) with Case 2 (Inputs are Discharging Capacity and CVCC)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 45ms/step - loss: 2.5293 - val_loss: 1.9650
Epoch 2/10000
5/5 [==============================] - 0s 10ms/step - loss: 1.7175 - val_loss: 1.3276
Epoch 3/10000
5/5 [==============================] - 0s 15ms/step - loss: 1.2260 - val_loss: 0.8175
Epoch 4/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.8031 - val_loss: 0.4565
Epoch 5/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.5273 - val_loss: 0.2242
Epoch 6/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.4498 - val_loss: 0.1019
Epoch 7/10000
5/5 [==============================] - 0s 13ms/step - loss: 0.3727 - val_loss: 0.0586
Epoch 8/10000
5/5 [==============================] - 0s 9ms/step - loss: 0.3050 - val_loss: 0.0502
Epoch 9/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.3069 - val_loss: 0.0457
Epoch 10/10000
5/5 [==============================] - 0s 13ms/step - loss: 0.2808 - val_loss: 0.0472


In [51]:
## Convolutional Neural Network (CNN) with Case 3 (Inputs are Discharging Capacity, CCCT, and CCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 49ms/step - loss: 3.2231 - val_loss: 2.0602
Epoch 2/10000
5/5 [==============================] - 0s 14ms/step - loss: 2.5673 - val_loss: 1.5706
Epoch 3/10000
5/5 [==============================] - 0s 14ms/step - loss: 2.1356 - val_loss: 1.2069
Epoch 4/10000
5/5 [==============================] - 0s 14ms/step - loss: 1.7299 - val_loss: 0.9148
Epoch 5/10000
5/5 [==============================] - 0s 14ms/step - loss: 1.3507 - val_loss: 0.6917
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 1.0559 - val_loss: 0.5207
Epoch 7/10000
5/5 [==============================] - 0s 13ms/step - loss: 0.7015 - val_loss: 0.4093
Epoch 8/10000
5/5 [==============================] - 0s 9ms/step - loss: 0.5221 - val_loss: 0.3617
Epoch 9/10000
5/5 [==============================] - 0s 13ms/step - loss: 0.3376 - val_loss: 0.3644
Epoch 10/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.3088 - val_loss: 0.3694


In [72]:
## Convolutional Neural Network (CNN) with Case 4 (Inputs are Discharging Capacity, CCCT, CCV, and CVCC)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 51ms/step - loss: 2.4409 - val_loss: 1.2175
Epoch 2/10000
5/5 [==============================] - 0s 10ms/step - loss: 1.1516 - val_loss: 0.4494
Epoch 3/10000
5/5 [==============================] - 0s 13ms/step - loss: 0.5526 - val_loss: 0.2208
Epoch 4/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.4825 - val_loss: 0.2156
Epoch 5/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.3687 - val_loss: 0.1746
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.2516 - val_loss: 0.1617
Epoch 7/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.2821 - val_loss: 0.2163
Epoch 8/10000
5/5 [==============================] - 0s 11ms/step - loss: 0.1612 - val_loss: 0.2716
Epoch 9/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.1802 - val_loss: 0.2195
Epoch 10/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.1346 - val_loss: 0.1582

In [97]:
## Convolutional Neural Network (CNN) with Case 5 (Inputs are CCDT and CCDV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 45ms/step - loss: 2.2278 - val_loss: 1.5452
Epoch 2/10000
5/5 [==============================] - 0s 9ms/step - loss: 1.6479 - val_loss: 0.8358
Epoch 3/10000
5/5 [==============================] - 0s 14ms/step - loss: 1.2350 - val_loss: 0.3869
Epoch 4/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.9605 - val_loss: 0.1858
Epoch 5/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.8295 - val_loss: 0.3093
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.6137 - val_loss: 0.8613
Epoch 7/10000
5/5 [==============================] - 0s 9ms/step - loss: 0.4179 - val_loss: 1.8097
Epoch 8/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.3512 - val_loss: 3.0304
Epoch 9/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.2809 - val_loss: 4.0681
Epoch 10/10000
5/5 [==============================] - 0s 9ms/step - loss: 0.2638 - val_loss: 4.8143
Ep

In [112]:
## Convolutional Neural Network (CNN) with Case 6 (Inputs are CCDV and RoCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 67ms/step - loss: 2.6784 - val_loss: 1.8838
Epoch 2/10000
5/5 [==============================] - 0s 19ms/step - loss: 1.7860 - val_loss: 0.8977
Epoch 3/10000
5/5 [==============================] - 0s 15ms/step - loss: 1.1518 - val_loss: 0.2957
Epoch 4/10000
5/5 [==============================] - 0s 21ms/step - loss: 0.7459 - val_loss: 0.1195
Epoch 5/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.5527 - val_loss: 0.3504
Epoch 6/10000
5/5 [==============================] - 0s 15ms/step - loss: 0.4170 - val_loss: 0.8738
Epoch 7/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.3417 - val_loss: 1.5211
Epoch 8/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.2447 - val_loss: 2.0198
Epoch 9/10000
5/5 [==============================] - 0s 14ms/step - loss: 0.2013 - val_loss: 2.5607
Epoch 10/10000
5/5 [==============================] - 0s 15ms/step - loss: 0.1561 - val_loss: 3.1068

In [136]:
## Convolutional Neural Network (CNN) with Case 7 (Inputs are Discharging Capacity, CCDT, and CCDV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 50ms/step - loss: 2.7812 - val_loss: 3.0869
Epoch 2/10000
5/5 [==============================] - 0s 15ms/step - loss: 2.1964 - val_loss: 2.0998
Epoch 3/10000
5/5 [==============================] - 0s 14ms/step - loss: 1.7504 - val_loss: 1.2881
Epoch 4/10000
5/5 [==============================] - 0s 15ms/step - loss: 1.3820 - val_loss: 0.6845
Epoch 5/10000
5/5 [==============================] - 0s 10ms/step - loss: 1.0820 - val_loss: 0.2959
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.8299 - val_loss: 0.2048
Epoch 7/10000
5/5 [==============================] - 0s 9ms/step - loss: 0.5930 - val_loss: 0.5251
Epoch 8/10000
5/5 [==============================] - 0s 9ms/step - loss: 0.4080 - val_loss: 1.2781
Epoch 9/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.3469 - val_loss: 2.3682
Epoch 10/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.2813 - val_loss: 3.5064
E

In [151]:
## Convolutional Neural Network (CNN) with Case 8 (Inputs are Discharging Capacity, CCDT, CCDV, and RoCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),  # Reduced filter size and kernel size
    MaxPooling1D(pool_size=2),  # Standard pooling size
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),  # Slightly less dropout to keep more information
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 1s 72ms/step - loss: 2.3763 - val_loss: 1.4429
Epoch 2/10000
5/5 [==============================] - 0s 21ms/step - loss: 1.4022 - val_loss: 0.6912
Epoch 3/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.8026 - val_loss: 0.1801
Epoch 4/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.5536 - val_loss: 0.0599
Epoch 5/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.4266 - val_loss: 0.2920
Epoch 6/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.3236 - val_loss: 0.7069
Epoch 7/10000
5/5 [==============================] - 0s 19ms/step - loss: 0.2214 - val_loss: 1.1173
Epoch 8/10000
5/5 [==============================] - 0s 18ms/step - loss: 0.1345 - val_loss: 1.5355
Epoch 9/10000
5/5 [==============================] - 0s 17ms/step - loss: 0.1405 - val_loss: 1.8892
Epoch 10/10000
5/5 [==============================] - 0s 20ms/step - loss: 0.1388 - val_loss: 2.0520

In [164]:
## Long Short-Term Memory (LSTM) Model with Case 1 (Inputs are CCCT and CCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 11s 431ms/step - loss: 2.8989 - val_loss: 2.9589
Epoch 2/10000
5/5 [==============================] - 0s 27ms/step - loss: 2.3237 - val_loss: 2.7653
Epoch 3/10000
5/5 [==============================] - 0s 24ms/step - loss: 1.4998 - val_loss: 2.6660
Epoch 4/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.8724 - val_loss: 2.4294
Epoch 5/10000
5/5 [==============================] - 0s 25ms/step - loss: 0.8580 - val_loss: 2.0207
Epoch 6/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.4868 - val_loss: 1.5548
Epoch 7/10000
5/5 [==============================] - 0s 23ms/step - loss: 0.3929 - val_loss: 0.9611
Epoch 8/10000
5/5 [==============================] - 0s 24ms/step - loss: 0.2432 - val_loss: 0.3340
Epoch 9/10000
5/5 [==============================] - 0s 24ms/step - loss: 0.0765 - val_loss: 0.0389
Epoch 10/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.0947 - val_loss: 0.04

In [180]:
## Long Short-Term Memory (LSTM) Model with Case 2 (Inputs are Discharging Capacity and CVCC)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/100
5/5 [==============================] - 11s 411ms/step - loss: 2.8008 - val_loss: 2.3044
Epoch 2/100
5/5 [==============================] - 0s 24ms/step - loss: 2.1094 - val_loss: 1.5041
Epoch 3/100
5/5 [==============================] - 0s 24ms/step - loss: 1.3230 - val_loss: 0.6045
Epoch 4/100
5/5 [==============================] - 0s 25ms/step - loss: 0.5341 - val_loss: 0.0684
Epoch 5/100
5/5 [==============================] - 0s 22ms/step - loss: 0.2976 - val_loss: 0.3925
Epoch 6/100
5/5 [==============================] - 0s 27ms/step - loss: 0.3757 - val_loss: 0.0612
Epoch 7/100
5/5 [==============================] - 0s 23ms/step - loss: 0.1801 - val_loss: 0.0651
Epoch 8/100
5/5 [==============================] - 0s 24ms/step - loss: 0.1354 - val_loss: 0.1275
Epoch 9/100
5/5 [==============================] - 0s 23ms/step - loss: 0.1138 - val_loss: 0.0461
Epoch 10/100
5/5 [==============================] - 0s 27ms/step - loss: 0.0410 - val_loss: 0.0072
Epoch 11/100
5/5 

In [196]:
## Long Short-Term Memory (LSTM) with Case 3 (Inputs are Discharging Capacity, CCCT, and CCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 12s 468ms/step - loss: 3.0303 - val_loss: 2.9764
Epoch 2/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.6545 - val_loss: 2.8707
Epoch 3/10000
5/5 [==============================] - 0s 28ms/step - loss: 2.1318 - val_loss: 2.7647
Epoch 4/10000
5/5 [==============================] - 0s 29ms/step - loss: 1.3485 - val_loss: 2.6873
Epoch 5/10000
5/5 [==============================] - 0s 30ms/step - loss: 0.6833 - val_loss: 2.5112
Epoch 6/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.7505 - val_loss: 2.1682
Epoch 7/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.4067 - val_loss: 1.7052
Epoch 8/10000
5/5 [==============================] - 0s 27ms/step - loss: 0.3759 - val_loss: 1.1200
Epoch 9/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.2156 - val_loss: 0.4879
Epoch 10/10000
5/5 [==============================] - 0s 29ms/step - loss: 0.1265 - val_loss: 0.11

In [202]:
## Long Short-Term Memory (LSTM) Model with Case 4 (Inputs are Discharging Capacity, CCCT, CCV, and CVCC)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 11s 636ms/step - loss: 2.9330 - val_loss: 2.8654
Epoch 2/10000
5/5 [==============================] - 0s 55ms/step - loss: 2.3298 - val_loss: 2.6846
Epoch 3/10000
5/5 [==============================] - 0s 58ms/step - loss: 1.5654 - val_loss: 2.3622
Epoch 4/10000
5/5 [==============================] - 0s 70ms/step - loss: 0.7783 - val_loss: 1.6363
Epoch 5/10000
5/5 [==============================] - 0s 40ms/step - loss: 0.4294 - val_loss: 0.4123
Epoch 6/10000
5/5 [==============================] - 0s 31ms/step - loss: 0.1901 - val_loss: 0.1279
Epoch 7/10000
5/5 [==============================] - 0s 34ms/step - loss: 0.1587 - val_loss: 0.0639
Epoch 8/10000
5/5 [==============================] - 0s 32ms/step - loss: 0.0751 - val_loss: 0.0191
Epoch 9/10000
5/5 [==============================] - 0s 33ms/step - loss: 0.0598 - val_loss: 0.0258
Epoch 10/10000
5/5 [==============================] - 0s 33ms/step - loss: 0.0504 - val_loss: 0.05

In [209]:
## Long Short-Term Memory (LSTM) Model with with Case 5 (Inputs are CCDT and CCDV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 11s 585ms/step - loss: 2.9368 - val_loss: 2.7507
Epoch 2/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.3494 - val_loss: 2.2842
Epoch 3/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.7416 - val_loss: 1.7777
Epoch 4/10000
5/5 [==============================] - 0s 24ms/step - loss: 1.3582 - val_loss: 0.9481
Epoch 5/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.1255 - val_loss: 0.2216
Epoch 6/10000
5/5 [==============================] - 0s 23ms/step - loss: 0.6345 - val_loss: 0.5868
Epoch 7/10000
5/5 [==============================] - 0s 23ms/step - loss: 0.2335 - val_loss: 3.7560
Epoch 8/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.1069 - val_loss: 5.3163
Epoch 9/10000
5/5 [==============================] - 0s 25ms/step - loss: 0.1560 - val_loss: 3.2295
Epoch 10/10000
5/5 [==============================] - 0s 27ms/step - loss: 0.0666 - val_loss: 1.60

In [221]:
## Long Short-Term Memory (LSTM) Model with Case 6 (Inputs are CCDV and RoCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 11s 432ms/step - loss: 2.9757 - val_loss: 2.9421
Epoch 2/10000
5/5 [==============================] - 0s 29ms/step - loss: 2.3998 - val_loss: 2.7969
Epoch 3/10000
5/5 [==============================] - 0s 30ms/step - loss: 1.7193 - val_loss: 2.6994
Epoch 4/10000
5/5 [==============================] - 0s 28ms/step - loss: 1.0196 - val_loss: 2.3529
Epoch 5/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.8311 - val_loss: 1.5874
Epoch 6/10000
5/5 [==============================] - 0s 27ms/step - loss: 0.3790 - val_loss: 0.6229
Epoch 7/10000
5/5 [==============================] - 0s 30ms/step - loss: 0.1780 - val_loss: 0.0851
Epoch 8/10000
5/5 [==============================] - 0s 31ms/step - loss: 0.1688 - val_loss: 0.0180
Epoch 9/10000
5/5 [==============================] - 0s 27ms/step - loss: 0.0786 - val_loss: 0.0727
Epoch 10/10000
5/5 [==============================] - 0s 29ms/step - loss: 0.0804 - val_loss: 0.06

In [240]:
## Long Short-Term Memory (LSTM) Model with Case 7 (Inputs are Discharging Capacity, CCDT, and CCDV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 12s 443ms/step - loss: 2.8694 - val_loss: 2.6368
Epoch 2/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.2094 - val_loss: 2.1976
Epoch 3/10000
5/5 [==============================] - 0s 26ms/step - loss: 1.5268 - val_loss: 1.6744
Epoch 4/10000
5/5 [==============================] - 0s 25ms/step - loss: 1.2521 - val_loss: 1.0244
Epoch 5/10000
5/5 [==============================] - 0s 30ms/step - loss: 1.0393 - val_loss: 0.3812
Epoch 6/10000
5/5 [==============================] - 0s 32ms/step - loss: 0.5903 - val_loss: 0.4264
Epoch 7/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.2375 - val_loss: 3.1841
Epoch 8/10000
5/5 [==============================] - 0s 24ms/step - loss: 0.2138 - val_loss: 3.5671
Epoch 9/10000
5/5 [==============================] - 0s 25ms/step - loss: 0.1170 - val_loss: 1.2852
Epoch 10/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.0876 - val_loss: 0.62

In [247]:
## Long Short-Term Memory (LSTM) Model with Case 8 (Inputs are Discharging Capacity, CCDT, CCDV, and RoCV)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Flatten inputs for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
X_test_flat = X_test.reshape(-1, X_test.shape[-1])

# Standardize the features
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

# Reshape for CNN input
X_train_scaled = X_train_scaled_flat.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM model
model = Sequential([
    Bidirectional(LSTM(50, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=10000, batch_size=32, callbacks=[early_stopping])

# Predict and evaluate the model
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/10000
5/5 [==============================] - 12s 449ms/step - loss: 2.8286 - val_loss: 2.7357
Epoch 2/10000
5/5 [==============================] - 0s 33ms/step - loss: 2.0705 - val_loss: 2.3482
Epoch 3/10000
5/5 [==============================] - 0s 33ms/step - loss: 1.2631 - val_loss: 1.5847
Epoch 4/10000
5/5 [==============================] - 0s 37ms/step - loss: 0.6975 - val_loss: 0.7126
Epoch 5/10000
5/5 [==============================] - 0s 35ms/step - loss: 0.3816 - val_loss: 0.1552
Epoch 6/10000
5/5 [==============================] - 0s 33ms/step - loss: 0.2881 - val_loss: 0.1227
Epoch 7/10000
5/5 [==============================] - 0s 34ms/step - loss: 0.1914 - val_loss: 0.0513
Epoch 8/10000
5/5 [==============================] - 0s 35ms/step - loss: 0.0642 - val_loss: 0.0087
Epoch 9/10000
5/5 [==============================] - 0s 34ms/step - loss: 0.0714 - val_loss: 0.0116
Epoch 10/10000
5/5 [==============================] - 0s 41ms/step - loss: 0.0489 - val_loss: 0.06

In [250]:
## Transformer Model with Case 1 (Inputs are CCCT and CCV)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Combine and reshape data
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 89ms/step - loss: 0.9831 - mae: 0.9730 - val_loss: 0.6405 - val_mae: 0.7964
Epoch 2/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.6498 - mae: 0.7924 - val_loss: 0.6321 - val_mae: 0.7914
Epoch 3/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6361 - mae: 0.7836 - val_loss: 0.6307 - val_mae: 0.7907
Epoch 4/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6297 - mae: 0.7795 - val_loss: 0.6286 - val_mae: 0.7893
Epoch 5/1000
5/5 [==============================] - 0s 20ms/step - loss: 0.6266 - mae: 0.7776 - val_loss: 0.6260 - val_mae: 0.7877
Epoch 6/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6240 - mae: 0.7759 - val_loss: 0.6235 - val_mae: 0.7861
Epoch 7/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6236 - mae: 0.7757 - val_loss: 0.6215 - val_mae: 0.7849
Epoch 8/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6207 - m

In [252]:
## Transformer Model with Case 2 (Inputs are Discharging Capacity and CVCC)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 87ms/step - loss: 0.8770 - mae: 0.9055 - val_loss: 0.6579 - val_mae: 0.8078
Epoch 2/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6603 - mae: 0.7992 - val_loss: 0.6511 - val_mae: 0.8035
Epoch 3/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6550 - mae: 0.7957 - val_loss: 0.6484 - val_mae: 0.8018
Epoch 4/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6500 - mae: 0.7925 - val_loss: 0.6456 - val_mae: 0.8001
Epoch 5/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6471 - mae: 0.7907 - val_loss: 0.6429 - val_mae: 0.7984
Epoch 6/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6432 - mae: 0.7882 - val_loss: 0.6406 - val_mae: 0.7970
Epoch 7/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6398 - mae: 0.7861 - val_loss: 0.6384 - val_mae: 0.7956
Epoch 8/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6379 - m

In [253]:
## Transformer Model with Case 3 (Inputs are Discharging Capacity, CCCT, and CCV)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 84ms/step - loss: 1.6064 - mae: 1.2413 - val_loss: 0.9530 - val_mae: 0.9713
Epoch 2/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.9025 - mae: 0.9319 - val_loss: 0.7652 - val_mae: 0.8712
Epoch 3/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.7719 - mae: 0.8640 - val_loss: 0.7220 - val_mae: 0.8464
Epoch 4/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.7249 - mae: 0.8381 - val_loss: 0.7084 - val_mae: 0.8384
Epoch 5/1000
5/5 [==============================] - 0s 16ms/step - loss: 0.7147 - mae: 0.8317 - val_loss: 0.7051 - val_mae: 0.8365
Epoch 6/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.7065 - mae: 0.8273 - val_loss: 0.7032 - val_mae: 0.8353
Epoch 7/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.7038 - mae: 0.8256 - val_loss: 0.6991 - val_mae: 0.8329
Epoch 8/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6967 - m

In [254]:
## Transformer Model with Case 4 (Inputs are Discharging Capacity, CCCT, CCV, and CVCC)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccvB0005, X_train_ccvB0006], axis=0),
    np.concatenate([X_train_ccctB0005, X_train_ccctB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_cvccB0005, X_train_cvccB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccvB0005, X_test_ccvB0006], axis=0),
    np.concatenate([X_test_ccctB0005, X_test_ccctB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_cvccB0005, X_test_cvccB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 89ms/step - loss: 0.7413 - mae: 0.8459 - val_loss: 0.6043 - val_mae: 0.7739
Epoch 2/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6020 - mae: 0.7617 - val_loss: 0.6000 - val_mae: 0.7710
Epoch 3/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.5998 - mae: 0.7602 - val_loss: 0.5996 - val_mae: 0.7708
Epoch 4/1000
5/5 [==============================] - 0s 16ms/step - loss: 0.5993 - mae: 0.7599 - val_loss: 0.5994 - val_mae: 0.7707
Epoch 5/1000
5/5 [==============================] - 0s 16ms/step - loss: 0.5989 - mae: 0.7596 - val_loss: 0.5989 - val_mae: 0.7704
Epoch 6/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.5983 - mae: 0.7592 - val_loss: 0.5986 - val_mae: 0.7701
Epoch 7/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.5979 - mae: 0.7589 - val_loss: 0.5982 - val_mae: 0.7699
Epoch 8/1000
5/5 [==============================] - 0s 16ms/step - loss: 0.5974 - m

In [255]:
## Transformer Model with Case 5 (CCDT and CCDV)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 90ms/step - loss: 1.2770 - mae: 1.1060 - val_loss: 0.7384 - val_mae: 0.8560
Epoch 2/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.7596 - mae: 0.8579 - val_loss: 0.7241 - val_mae: 0.8478
Epoch 3/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.7302 - mae: 0.8414 - val_loss: 0.7194 - val_mae: 0.8450
Epoch 4/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.7202 - mae: 0.8357 - val_loss: 0.7142 - val_mae: 0.8419
Epoch 5/1000
5/5 [==============================] - 0s 21ms/step - loss: 0.7150 - mae: 0.8324 - val_loss: 0.7083 - val_mae: 0.8384
Epoch 6/1000
5/5 [==============================] - 0s 20ms/step - loss: 0.7070 - mae: 0.8278 - val_loss: 0.7026 - val_mae: 0.8350
Epoch 7/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.7011 - mae: 0.8242 - val_loss: 0.6976 - val_mae: 0.8320
Epoch 8/1000
5/5 [==============================] - 0s 20ms/step - loss: 0.6973 - m

In [256]:
## Transformer Model with Case 6 (Input are CCDV and RoV)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 98ms/step - loss: 0.7831 - mae: 0.8705 - val_loss: 0.6268 - val_mae: 0.7884
Epoch 2/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6242 - mae: 0.7766 - val_loss: 0.6156 - val_mae: 0.7811
Epoch 3/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6171 - mae: 0.7717 - val_loss: 0.6127 - val_mae: 0.7793
Epoch 4/1000
5/5 [==============================] - 0s 16ms/step - loss: 0.6143 - mae: 0.7697 - val_loss: 0.6111 - val_mae: 0.7782
Epoch 5/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6116 - mae: 0.7679 - val_loss: 0.6098 - val_mae: 0.7774
Epoch 6/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6100 - mae: 0.7669 - val_loss: 0.6088 - val_mae: 0.7767
Epoch 7/1000
5/5 [==============================] - 0s 20ms/step - loss: 0.6089 - mae: 0.7662 - val_loss: 0.6078 - val_mae: 0.7761
Epoch 8/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6079 - m

In [257]:
## Transformer Model with Case 7 (Discharging Capacity, CCDT, and CCDV)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self-attention and normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network and normalization
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape[1]))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)  # Apply Dense to each timestep
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape for model input
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])

# If model outputs at each timestep and you need targets for each timestep:
y_train = np.repeat(y_train[:, np.newaxis], X_train.shape[1], axis=1)
y_test = np.repeat(y_test[:, np.newaxis], X_test.shape[1], axis=1)

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = build_model((X_train.shape[1], X_train.shape[2]), head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 3s 129ms/step - loss: 0.9356 - mae: 0.9445 - val_loss: 0.6145 - val_mae: 0.7804
Epoch 2/1000
5/5 [==============================] - 0s 26ms/step - loss: 0.6194 - mae: 0.7735 - val_loss: 0.6124 - val_mae: 0.7791
Epoch 3/1000
5/5 [==============================] - 0s 25ms/step - loss: 0.6133 - mae: 0.7690 - val_loss: 0.6115 - val_mae: 0.7785
Epoch 4/1000
5/5 [==============================] - 0s 25ms/step - loss: 0.6117 - mae: 0.7681 - val_loss: 0.6104 - val_mae: 0.7778
Epoch 5/1000
5/5 [==============================] - 0s 23ms/step - loss: 0.6100 - mae: 0.7669 - val_loss: 0.6093 - val_mae: 0.7771
Epoch 6/1000
5/5 [==============================] - 0s 22ms/step - loss: 0.6091 - mae: 0.7665 - val_loss: 0.6082 - val_mae: 0.7764
Epoch 7/1000
5/5 [==============================] - 0s 23ms/step - loss: 0.6076 - mae: 0.7653 - val_loss: 0.6072 - val_mae: 0.7757
Epoch 8/1000
5/5 [==============================] - 0s 26ms/step - loss: 0.6069 - 

In [258]:
## Transformer Model with Case 8 (Discharging Capacity, CCDT, RoCV, and CCDV)

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return ff

def build_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = layers.Input(shape=(None, input_shape))
    x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Combine training sets from B0005 and B0006
X_train = np.concatenate([
    np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006], axis=0),
    np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006], axis=0),
    np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006], axis=0),
    np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006], axis=0)
], axis=1)
y_train = np.concatenate([y_train_B0005, y_train_B0006], axis=0)

# Combine testing sets from B0005 and B0006
X_test = np.concatenate([
    np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006], axis=0),
    np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006], axis=0),
    np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006], axis=0),
    np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006], axis=0)
], axis=1)
y_test = np.concatenate([y_test_B0005, y_test_B0006], axis=0)

# Reshape data if necessary:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])  # Add a sequence dimension
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

model = build_model(X_train.shape[2], head_size=128, num_heads=4, ff_dim=512, dropout=0.1)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1000, callbacks=[early_stopping])

# Prediction and Evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 1/1000
5/5 [==============================] - 6s 136ms/step - loss: 1.0948 - mae: 0.9786 - val_loss: 0.6157 - val_mae: 0.7811
Epoch 2/1000
5/5 [==============================] - 0s 28ms/step - loss: 0.6248 - mae: 0.7757 - val_loss: 0.6138 - val_mae: 0.7800
Epoch 3/1000
5/5 [==============================] - 0s 28ms/step - loss: 0.6136 - mae: 0.7692 - val_loss: 0.6135 - val_mae: 0.7798
Epoch 4/1000
5/5 [==============================] - 0s 33ms/step - loss: 0.6136 - mae: 0.7691 - val_loss: 0.6120 - val_mae: 0.7788
Epoch 5/1000
5/5 [==============================] - 0s 26ms/step - loss: 0.6110 - mae: 0.7675 - val_loss: 0.6100 - val_mae: 0.7775
Epoch 6/1000
5/5 [==============================] - 0s 22ms/step - loss: 0.6094 - mae: 0.7664 - val_loss: 0.6082 - val_mae: 0.7764
Epoch 7/1000
5/5 [==============================] - 0s 27ms/step - loss: 0.6079 - mae: 0.7654 - val_loss: 0.6068 - val_mae: 0.7755
Epoch 8/1000
5/5 [==============================] - 0s 30ms/step - loss: 0.6065 - 

In [11]:
# Mamba Model with Case 1 (Inputs are CCCT and CCV)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

# Assuming bucketing into 1000 categories for each feature
num_buckets = 1000

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccv_encoded_train = np.floor(np.concatenate([X_train_ccvB0005, X_train_ccvB0006]) * num_buckets).astype(int) % num_buckets
ccct_encoded_train = np.floor(np.concatenate([X_train_ccctB0005, X_train_ccctB0006]) * num_buckets).astype(int) % num_buckets
ccv_encoded_test = np.floor(np.concatenate([X_test_ccvB0005, X_test_ccvB0006]) * num_buckets).astype(int) % num_buckets
ccct_encoded_test = np.floor(np.concatenate([X_test_ccctB0005, X_test_ccctB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([ccv_encoded_train, ccct_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([ccv_encoded_test, ccct_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

# Model adaptation
class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        volt_embeddings = self.embedding(features[:, :features.shape[1]//2])
        time_embeddings = self.embedding(features[:, features.shape[1]//2:])
        combined_embeddings = (volt_embeddings + time_embeddings) / 2
        outputs = self.mamba(inputs_embeds=combined_embeddings)
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Building and compiling the model
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Training function
def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training loop
for epoch in range(50):  # number of epochs
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)  # Directly using test as validation for demonstration
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.re

Epoch 0, Train Loss: 3.47023868560791, Test Loss: 0.46546128392219543
Epoch 1, Train Loss: 1.464688777923584, Test Loss: 0.22463059425354004
Epoch 2, Train Loss: 0.16540637612342834, Test Loss: 0.16351191699504852
Epoch 3, Train Loss: 0.7807360291481018, Test Loss: 0.12750554084777832
Epoch 4, Train Loss: 0.31307122111320496, Test Loss: 0.1263696253299713
Epoch 5, Train Loss: 0.0947418361902237, Test Loss: 0.1362079232931137
Epoch 6, Train Loss: 0.24101115763187408, Test Loss: 0.11316194385290146
Epoch 7, Train Loss: 0.19575975835323334, Test Loss: 0.11062703281641006
Epoch 8, Train Loss: 0.0911204069852829, Test Loss: 0.155417799949646
Epoch 9, Train Loss: 0.0999346524477005, Test Loss: 0.20099937915802002
Epoch 10, Train Loss: 0.12404144555330276, Test Loss: 0.21187099814414978
Epoch 11, Train Loss: 0.11124246567487717, Test Loss: 0.19787561893463135
Epoch 12, Train Loss: 0.09610212594270706, Test Loss: 0.18037913739681244
Epoch 13, Train Loss: 0.08479943126440048, Test Loss: 0.16970

In [12]:
# Mamba model with Case 2 (Inputs are Discharging Capacity and CVCC)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

# Assuming bucketing into 1000 categories for each feature
num_buckets = 1000

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]


# Encoding and reshaping
dcinput_encoded_train = np.floor(np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_test = np.floor(np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006]) * num_buckets).astype(int) % num_buckets
cvcc_encoded_train = np.floor(np.concatenate([X_train_cvccB0005, X_train_cvccB0006]) * num_buckets).astype(int) % num_buckets
cvcc_encoded_test = np.floor(np.concatenate([X_test_cvccB0005, X_test_cvccB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([dcinput_encoded_train, cvcc_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([dcinput_encoded_test, cvcc_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

# Model adaptation
class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        volt_embeddings = self.embedding(features[:, :features.shape[1]//2])
        time_embeddings = self.embedding(features[:, features.shape[1]//2:])
        combined_embeddings = (volt_embeddings + time_embeddings) / 2
        outputs = self.mamba(inputs_embeds=combined_embeddings)
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Building and compiling the model
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Training function
def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training loop
for epoch in range(50):  # number of epochs
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)  # Directly using test as validation for demonstration
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Train Loss: 3.5249924659729004, Test Loss: 18.851703643798828
Epoch 1, Train Loss: 15.458888053894043, Test Loss: 0.24435439705848694
Epoch 2, Train Loss: 0.16891461610794067, Test Loss: 3.648308038711548
Epoch 3, Train Loss: 4.349555492401123, Test Loss: 0.8122110962867737
Epoch 4, Train Loss: 1.135394811630249, Test Loss: 0.5162286758422852
Epoch 5, Train Loss: 0.22861434519290924, Test Loss: 1.805713415145874
Epoch 6, Train Loss: 1.047043800354004, Test Loss: 1.3872369527816772
Epoch 7, Train Loss: 0.5983670949935913, Test Loss: 0.533351719379425
Epoch 8, Train Loss: 0.13417048752307892, Test Loss: 0.36086857318878174
Epoch 9, Train Loss: 0.46654626727104187, Test Loss: 0.2023242563009262
Epoch 10, Train Loss: 0.31373757123947144, Test Loss: 0.15207701921463013
Epoch 11, Train Loss: 0.11403529345989227, Test Loss: 0.256428599357605
Epoch 12, Train Loss: 0.07690814882516861, Test Loss: 0.3894669711589813
Epoch 13, Train Loss: 0.12579505145549774, Test Loss: 0.362702816724777

In [13]:
# Mamba model with Case 3 (Inputs are Discharging Capacity, CCCT, and CCV)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

num_buckets = 1000  # Assuming bucketing into 1000 categories for each feature

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccv_encoded_train = np.floor(np.concatenate([X_train_ccvB0005, X_train_ccvB0006]) * num_buckets).astype(int) % num_buckets
ccv_encoded_test = np.floor(np.concatenate([X_test_ccvB0005, X_test_ccvB0006 ]) * num_buckets).astype(int) % num_buckets
ccct_encoded_train = np.floor(np.concatenate([X_train_ccctB0005, X_train_ccctB0006]) * num_buckets).astype(int) % num_buckets
ccct_encoded_test = np.floor(np.concatenate([X_test_ccctB0005, X_test_ccctB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_train = np.floor(np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_test = np.floor(np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([dcinput_encoded_train, ccv_encoded_train, ccct_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([dcinput_encoded_test, ccv_encoded_test, ccct_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        embeddings = self.embedding(features)
        # Aggregate embeddings: average across the sequence length (here across all features)
        pooled_embeddings = embeddings.mean(dim=1)
        outputs = self.mamba(inputs_embeds=pooled_embeddings.unsqueeze(1))
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Initialize the model, optimizer, and loss function
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training and evaluation code setup
for epoch in range(50):
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Train Loss: 3.7678608894348145, Test Loss: 7.2793288230896
Epoch 1, Train Loss: 11.126029014587402, Test Loss: 0.3485347032546997
Epoch 2, Train Loss: 0.08733738958835602, Test Loss: 0.32789701223373413
Epoch 3, Train Loss: 1.6042956113815308, Test Loss: 0.4157648980617523
Epoch 4, Train Loss: 1.5305238962173462, Test Loss: 0.2247483730316162
Epoch 5, Train Loss: 0.5010473728179932, Test Loss: 0.16264133155345917
Epoch 6, Train Loss: 0.10663770139217377, Test Loss: 0.3307138979434967
Epoch 7, Train Loss: 0.448649525642395, Test Loss: 0.37511196732521057
Epoch 8, Train Loss: 0.3706232011318207, Test Loss: 0.23298300802707672
Epoch 9, Train Loss: 0.09525060653686523, Test Loss: 0.11024738848209381
Epoch 10, Train Loss: 0.11203030496835709, Test Loss: 0.08696716278791428
Epoch 11, Train Loss: 0.2501625418663025, Test Loss: 0.09667778015136719
Epoch 12, Train Loss: 0.2180679589509964, Test Loss: 0.10144173353910446
Epoch 13, Train Loss: 0.09354674816131592, Test Loss: 0.1167669072

In [14]:
# Mamba model with Case 4 (Discharging Capacity, CCCT, CCV, and CVC)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

num_buckets = 1000  # Assuming bucketing into 1000 categories for each feature

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccvB0005 = ccvB0005[0:103, :]
X_train_ccvB0006 = ccvB0006[0:60, :]
X_train_ccctB0005 = ccctB0005[0:103, :]
X_train_ccctB0006 = ccctB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_cvccB0005 = cvccB0005[0:103, :]
X_train_cvccB0006 = cvccB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccvB0005 = ccvB0005[104:, :]
X_test_ccvB0006 = ccvB0006[61:, :]
X_test_ccctB0005 = ccctB0005[104:, :]
X_test_ccctB0006 = ccctB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_cvccB0005 = cvccB0005[104:, :]
X_test_cvccB0006 = cvccB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccv_encoded_train = np.floor(np.concatenate([X_train_ccvB0005, X_train_ccvB0006]) * num_buckets).astype(int) % num_buckets
ccv_encoded_test = np.floor(np.concatenate([X_test_ccvB0005, X_test_ccvB0006 ]) * num_buckets).astype(int) % num_buckets
ccct_encoded_train = np.floor(np.concatenate([X_train_ccctB0005, X_train_ccctB0006]) * num_buckets).astype(int) % num_buckets
ccct_encoded_test = np.floor(np.concatenate([X_test_ccctB0005, X_test_ccctB0006]) * num_buckets).astype(int) % num_buckets
cvcc_encoded_train = np.floor(np.concatenate([X_train_cvccB0005, X_train_cvccB0006]) * num_buckets).astype(int) % num_buckets
cvcc_encoded_test = np.floor(np.concatenate([X_test_cvccB0005, X_test_cvccB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_train = np.floor(np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_test = np.floor(np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([dcinput_encoded_train, ccv_encoded_train, ccct_encoded_train, cvcc_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([dcinput_encoded_test, ccv_encoded_test, ccct_encoded_test, cvcc_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        embeddings = self.embedding(features)
        # Aggregate embeddings: average across the sequence length (here across all features)
        pooled_embeddings = embeddings.mean(dim=1)
        outputs = self.mamba(inputs_embeds=pooled_embeddings.unsqueeze(1))
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Initialize the model, optimizer, and loss function
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training and evaluation code setup
for epoch in range(50):
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Train Loss: 2.15959095954895, Test Loss: 37.701175689697266
Epoch 1, Train Loss: 32.655555725097656, Test Loss: 2.099595546722412
Epoch 2, Train Loss: 1.388293981552124, Test Loss: 4.543839454650879
Epoch 3, Train Loss: 5.376101016998291, Test Loss: 2.1483025550842285
Epoch 4, Train Loss: 3.265002489089966, Test Loss: 0.12268330156803131
Epoch 5, Train Loss: 0.6496732234954834, Test Loss: 1.1485528945922852
Epoch 6, Train Loss: 0.5741653442382812, Test Loss: 2.006399393081665
Epoch 7, Train Loss: 0.918693482875824, Test Loss: 1.2794015407562256
Epoch 8, Train Loss: 0.5839595794677734, Test Loss: 0.33368074893951416
Epoch 9, Train Loss: 0.21846389770507812, Test Loss: 0.07200238108634949
Epoch 10, Train Loss: 0.23989838361740112, Test Loss: 0.14988477528095245
Epoch 11, Train Loss: 0.36860936880111694, Test Loss: 0.12053872644901276
Epoch 12, Train Loss: 0.3147869408130646, Test Loss: 0.16232098639011383
Epoch 13, Train Loss: 0.18704017996788025, Test Loss: 0.3864569067955017
E

In [15]:
# Mamba Model with Case 5 (Inputs are CCDT and CCDV)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

num_buckets = 1000  # Assuming bucketing into 1000 categories for each feature

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccdt_encoded_train = np.floor(np.concatenate([X_train_ccdtB0005, X_train_ccdtB0006]) * num_buckets).astype(int) % num_buckets
ccdt_encoded_test = np.floor(np.concatenate([X_test_ccdtB0005, X_test_ccdtB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_train = np.floor(np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_test = np.floor(np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([ccdt_encoded_train, ccdv_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([ccdt_encoded_test, ccdv_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        embeddings = self.embedding(features)
        # Aggregate embeddings: average across the sequence length (here across all features)
        pooled_embeddings = embeddings.mean(dim=1)
        outputs = self.mamba(inputs_embeds=pooled_embeddings.unsqueeze(1))
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Initialize the model, optimizer, and loss function
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training and evaluation code setup
for epoch in range(50):
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Train Loss: 4.665439128875732, Test Loss: 13.348295211791992
Epoch 1, Train Loss: 17.630626678466797, Test Loss: 0.3846375346183777
Epoch 2, Train Loss: 0.20581316947937012, Test Loss: 1.626458764076233
Epoch 3, Train Loss: 3.3963441848754883, Test Loss: 1.0340076684951782
Epoch 4, Train Loss: 2.094491958618164, Test Loss: 0.19102731347084045
Epoch 5, Train Loss: 0.27074581384658813, Test Loss: 0.6760230660438538
Epoch 6, Train Loss: 0.5208434462547302, Test Loss: 0.9812687039375305
Epoch 7, Train Loss: 0.8450825214385986, Test Loss: 0.5116139054298401
Epoch 8, Train Loss: 0.35594642162323, Test Loss: 0.15284375846385956
Epoch 9, Train Loss: 0.06434759497642517, Test Loss: 0.12361475825309753
Epoch 10, Train Loss: 0.15865084528923035, Test Loss: 0.20304188132286072
Epoch 11, Train Loss: 0.31830650568008423, Test Loss: 0.19441728293895721
Epoch 12, Train Loss: 0.30452123284339905, Test Loss: 0.1219383254647255
Epoch 13, Train Loss: 0.17622046172618866, Test Loss: 0.074375145137

In [17]:
# Mamba model with Case 6 (Inputs are CCDV and RoCV)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

num_buckets = 1000  # Assuming bucketing into 1000 categories for each feature

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccdv_encoded_train = np.floor(np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_test = np.floor(np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_RoCV_encoded_train = np.floor(np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006]) * num_buckets).astype(int) % num_buckets
ccdv_RoCV_encoded_test = np.floor(np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([ccdt_encoded_train, ccdv_RoCV_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([ccdt_encoded_test, ccdv_RoCV_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        embeddings = self.embedding(features)
        # Aggregate embeddings: average across the sequence length (here across all features)
        pooled_embeddings = embeddings.mean(dim=1)
        outputs = self.mamba(inputs_embeds=pooled_embeddings.unsqueeze(1))
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Initialize the model, optimizer, and loss function
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training and evaluation code setup
for epoch in range(50):
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

Epoch 0, Train Loss: 3.2787327766418457, Test Loss: 13.351444244384766
Epoch 1, Train Loss: 16.174299240112305, Test Loss: 0.5206835865974426
Epoch 2, Train Loss: 0.23975305259227753, Test Loss: 1.591067910194397
Epoch 3, Train Loss: 3.25105357170105, Test Loss: 0.9696124792098999
Epoch 4, Train Loss: 2.0578887462615967, Test Loss: 0.22061271965503693
Epoch 5, Train Loss: 0.33194202184677124, Test Loss: 0.7270879745483398
Epoch 6, Train Loss: 0.4166552424430847, Test Loss: 1.15664541721344
Epoch 7, Train Loss: 0.8217693567276001, Test Loss: 0.7076021432876587
Epoch 8, Train Loss: 0.4199465811252594, Test Loss: 0.2356298863887787
Epoch 9, Train Loss: 0.08168550580739975, Test Loss: 0.12419198453426361
Epoch 10, Train Loss: 0.14479589462280273, Test Loss: 0.20941077172756195
Epoch 11, Train Loss: 0.3185305893421173, Test Loss: 0.22209101915359497
Epoch 12, Train Loss: 0.29179254174232483, Test Loss: 0.17910988628864288
Epoch 13, Train Loss: 0.15463580191135406, Test Loss: 0.1720657050609

In [18]:
# Mamba model with Case 7 (Inputs are Discharging Capacity, CCDT, and CCDV)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

num_buckets = 1000  # Assuming bucketing into 1000 categories for each feature

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccdt_encoded_train = np.floor(np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdt_encoded_test = np.floor(np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_train = np.floor(np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_test = np.floor(np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_train = np.floor(np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_test = np.floor(np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([ccdt_encoded_train, ccdv_encoded_train, dcinput_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([ccdt_encoded_test, ccdv_encoded_test, dcinput_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        embeddings = self.embedding(features)
        # Aggregate embeddings: average across the sequence length (here across all features)
        pooled_embeddings = embeddings.mean(dim=1)
        outputs = self.mamba(inputs_embeds=pooled_embeddings.unsqueeze(1))
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Initialize the model, optimizer, and loss function
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training and evaluation code setup
for epoch in range(50):
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Train Loss: 2.8898022174835205, Test Loss: 6.978864669799805
Epoch 1, Train Loss: 10.386606216430664, Test Loss: 0.22618518769741058
Epoch 2, Train Loss: 0.06357471644878387, Test Loss: 0.4145304262638092
Epoch 3, Train Loss: 1.8132790327072144, Test Loss: 0.5028293132781982
Epoch 4, Train Loss: 1.7091317176818848, Test Loss: 0.25591087341308594
Epoch 5, Train Loss: 0.6421957015991211, Test Loss: 0.15268303453922272
Epoch 6, Train Loss: 0.12966135144233704, Test Loss: 0.28624242544174194
Epoch 7, Train Loss: 0.3680260479450226, Test Loss: 0.326660692691803
Epoch 8, Train Loss: 0.45881491899490356, Test Loss: 0.19686931371688843
Epoch 9, Train Loss: 0.21493417024612427, Test Loss: 0.08844052255153656
Epoch 10, Train Loss: 0.061762336641550064, Test Loss: 0.0725468322634697
Epoch 11, Train Loss: 0.1046893373131752, Test Loss: 0.09950225055217743
Epoch 12, Train Loss: 0.19622847437858582, Test Loss: 0.10268502682447433
Epoch 13, Train Loss: 0.1898466795682907, Test Loss: 0.085574

In [19]:
# Mamba model with Case 8 (Inputs are Discharging Capacity, CCDT, CCDB, and RoCV)
import numpy as np
import torch
from torch import nn
from transformers import MambaConfig, MambaModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for the Mamba model
config = MambaConfig(
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
)

num_buckets = 1000  # Assuming bucketing into 1000 categories for each feature

# Prepare the features (X) and the target (y)
# Cycles 1 to 103 for B0005 and cycles 1 to 60 for B0006 are considered above 80% SOH condition
X_train_ccdtB0005 = ccdtB0005[0:103, :]
X_train_ccdtB0006 = ccdtB0006[0:60, :]
X_train_ccdvB0005 = ccdvB0005[0:103, :]
X_train_ccdvB0006 = ccdvB0006[0:60, :]
X_train_dcinputB0005 = dcInputB0005[0:103, :]
X_train_dcinputB0006 = dcInputB0006[0:60, :]
X_train_ccdv_rocB0005 = ccdv_roc_B0005[0:103, :]
X_train_ccdv_rocB0006 = ccdv_roc_B0006[0:60, :]
y_train_B0005 = dcB0005[0:103, :]
y_train_B0006 = dcB0006[0:60, :]

# Cycles 104 to 167 for B0005 and cycles 61 to 167 for B0006 are considered under 80% SOH condition
X_test_ccdtB0005 = ccdtB0005[104:, :]
X_test_ccdtB0006 = ccdtB0006[61:, :]
X_test_ccdvB0005 = ccdvB0005[104:, :]
X_test_ccdvB0006 = ccdvB0006[61:, :]
X_test_dcinputB0005 = dcInputB0005[104:, :]
X_test_dcinputB0006 = dcInputB0006[61:, :]
X_test_ccdv_rocB0005 = ccdv_roc_B0005[104:, :]
X_test_ccdv_rocB0006 = ccdv_roc_B0006[61:, :]
y_test_B0005 = dcB0005[104:, :]
y_test_B0006 = dcB0006[61:, :]

# Encoding and reshaping
ccdt_encoded_train = np.floor(np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdt_encoded_test = np.floor(np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_train = np.floor(np.concatenate([X_train_ccdvB0005, X_train_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_encoded_test = np.floor(np.concatenate([X_test_ccdvB0005, X_test_ccdvB0006]) * num_buckets).astype(int) % num_buckets
ccdv_roc_encoded_train = np.floor(np.concatenate([X_train_ccdv_rocB0005, X_train_ccdv_rocB0006]) * num_buckets).astype(int) % num_buckets
ccdv_roc_encoded_test = np.floor(np.concatenate([X_test_ccdv_rocB0005, X_test_ccdv_rocB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_train = np.floor(np.concatenate([X_train_dcinputB0005, X_train_dcinputB0006]) * num_buckets).astype(int) % num_buckets
dcinput_encoded_test = np.floor(np.concatenate([X_test_dcinputB0005, X_test_dcinputB0006]) * num_buckets).astype(int) % num_buckets

# Convert to PyTorch tensors
X_train = torch.tensor(np.concatenate([ccdt_encoded_train, ccdv_encoded_train, dcinput_encoded_train, ccdv_roc_encoded_train], axis=1), dtype=torch.long)
y_train = torch.tensor(np.concatenate([y_train_B0005, y_train_B0006]), dtype=torch.float32)
X_test = torch.tensor(np.concatenate([ccdt_encoded_test, ccdv_encoded_test, dcinput_encoded_test, ccdv_roc_encoded_test], axis=1), dtype=torch.long)
y_test = torch.tensor(np.concatenate([y_test_B0005, y_test_B0006]), dtype=torch.float32)

class RegressionMamba(nn.Module):
    def __init__(self, config, num_buckets):
        super().__init__()
        self.embedding = nn.Embedding(num_buckets, config.hidden_size)
        self.mamba = MambaModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)  # Output layer for regression

    def forward(self, features):
        embeddings = self.embedding(features)
        # Aggregate embeddings: average across the sequence length (here across all features)
        pooled_embeddings = embeddings.mean(dim=1)
        outputs = self.mamba(inputs_embeds=pooled_embeddings.unsqueeze(1))
        regression_output = self.regressor(outputs.last_hidden_state[:, -1])
        return regression_output.squeeze()

# Initialize the model, optimizer, and loss function
model = RegressionMamba(config, num_buckets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = loss_fn(predictions, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
        loss = loss_fn(predictions, y)
        return loss, predictions

# Training and evaluation code setup
for epoch in range(50):
    train_loss = train()
    val_loss, _ = evaluate(X_test, y_test)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Test Loss: {val_loss}")

# Testing and metric evaluation
test_loss, predictions = evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Calculate MSE, MAE, and R2 score
predictions_np = predictions.detach().numpy()
y_test_np = y_test.detach().numpy()
mse = mean_squared_error(y_test_np, predictions_np)
mae = mean_absolute_error(y_test_np, predictions_np)
r2 = r2_score(y_test_np, predictions_np)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([163, 1])) that is different to the input size (torch.Size([163])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([169, 1])) that is different to the input size (torch.Size([169])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Train Loss: 3.7964632511138916, Test Loss: 9.38429069519043
Epoch 1, Train Loss: 13.24953556060791, Test Loss: 0.33392995595932007
Epoch 2, Train Loss: 0.09004606306552887, Test Loss: 0.3703033924102783
Epoch 3, Train Loss: 1.6288996934890747, Test Loss: 0.4809814393520355
Epoch 4, Train Loss: 1.5751744508743286, Test Loss: 0.2334008365869522
Epoch 5, Train Loss: 0.5389513373374939, Test Loss: 0.15834426879882812
Epoch 6, Train Loss: 0.11232025176286697, Test Loss: 0.3438052535057068
Epoch 7, Train Loss: 0.4404066801071167, Test Loss: 0.32538384199142456
Epoch 8, Train Loss: 0.3810332715511322, Test Loss: 0.1509667932987213
Epoch 9, Train Loss: 0.10194143652915955, Test Loss: 0.0833381935954094
Epoch 10, Train Loss: 0.10461597889661789, Test Loss: 0.11791308224201202
Epoch 11, Train Loss: 0.24323053658008575, Test Loss: 0.1361486315727234
Epoch 12, Train Loss: 0.23960311710834503, Test Loss: 0.12063427269458771
Epoch 13, Train Loss: 0.12763412296772003, Test Loss: 0.1091285273